In [ ]:
import pandas as pd

melb_data = pd.read_csv('data/melb_data_ps.csv', sep=',')
melb_df = melb_data.copy()
melb_df.head()

In [ ]:
melb_df.info()

In [ ]:
# Количество уникальных значений в столбце:

print(melb_df['Address'].nunique())

Итак, адрес строится следующим образом: сначала указывается номер дома и корпус, после указывается название улицы, а в конце — подтип улицы, но в некоторых случаях к подтипу добавляется географическая отметка (N — север, S — юг и т. д.), она нам не нужна . Для того чтобы выделить подтип улицы, на которой находится объект, можно использовать следующую функцию:

In [ ]:
# На вход данной функции поступает строка с адресом.
def get_street_type(address):
# Создаём список географических пометок exclude_list.
    exclude_list = ['N', 'S', 'W', 'E']
# Метод split() разбивает строку на слова по пробелу.
# В результате получаем список слов в строке и заносим его в переменную address_list.
    address_list = address.split(' ')
# Обрезаем список, оставляя в нём только последний элемент,
# потенциальный подтип улицы, и заносим в переменную street_type.
    street_type = address_list[-1]
# Делаем проверку на то, что полученный подтип является географической пометкой.
# Для этого проверяем его на наличие в списке exclude_list.
    if street_type in exclude_list:
# Если переменная street_type является географической пометкой,
# переопределяем её на второй элемент с конца списка address_list.
        street_type = address_list[-2]
# Возвращаем переменную street_type, в которой хранится подтип улицы.
    return street_type

Теперь применим эту функцию к столбцу c адресом. Для этого передадим функцию get_street_type в аргумент метода столбца apply(). В результате получим объект Series, который положим в переменную street_types:

In [ ]:
street_types = melb_df['Address'].apply(get_street_type)
display(street_types)

In [ ]:
display(street_types.value_counts())

In [ ]:
popular_stypes = street_types.value_counts().nlargest(10).index
print(popular_stypes)
# Index(['St', 'Rd', 'Ct', 'Dr', 'Av', 'Gr', 'Pde', 'Pl', 'Cr', 'Cl'], dtype='object')

Теперь, когда у нас есть список наиболее популярных подтипов улиц, введём lambda-функцию, которая будет проверять, есть ли строка x в этом перечне, и, если это так, lambda-функция будет возвращать x, в противном случае она будет возвращать строку 'other'. Наконец, применим такую функцию к Series street_types, полученной ранее, а результат определим в новый столбец таблицы StreetType:

In [ ]:
melb_df['StreetType'] = street_types.apply(lambda x: x if x in popular_stypes else 'other')
display(melb_df['StreetType'])

In [ ]:
melb_df = melb_df.drop('Address', axis=1)

Напишите функцию get_weekend(weekday), которая принимает на вход элемент столбца WeekdaySale и возвращает 1, если день является выходным, и 0 — в противном случае, и создайте столбец Weekend в таблице melb_df с помощью неё.

Примените эту функцию к столбцу и вычислите среднюю цену объекта недвижимости, продан

In [ ]:
melb_df['Date'] = pd.to_datetime(melb_df['Date'], dayfirst=True)
melb_df['WeekdaySale'] = melb_df['Date'].dt.dayofweek

def get_weekend(weekday):
    if weekday == 6 or weekday == 5:
        return 1
    return 0

melb_df['Weekend'] = melb_df['WeekdaySale'].apply(get_weekend)
round(melb_df[melb_df['Weekend'] == 1]['Price'].mean())

Преобразуйте столбец SellerG с наименованиями риелторских компаний в таблице melb_df следующим образом: оставьте в столбце только 49 самых популярных компаний, а остальные обозначьте как 'other'.
Найдите, во сколько раз минимальная цена объектов недвижимости, проданных компанией 'Nelson', больше минимальной цены объектов, проданных компаниями, обозначенными как 'other'. Ответ округлите до десятых.

In [ ]:
popular_seller = melb_df['SellerG'].value_counts().nlargest(49).index

melb_df['SellerType'] = melb_df['SellerG'].apply(lambda x: x if x in popular_seller else 'other')
a = melb_df[melb_df['SellerType'] == 'Nelson']['Price'].min()
b = melb_df[melb_df['SellerType'] == 'other']['Price'].min()
print(round(a/b,2))


In [ ]:
test = pd.Series([
    'Опыт работы 8 лет 3 месяца',
    'Опыт работы 3 года 5 месяцев',
    'Опыт работы 1 год 9 месяцев',
    'Опыт работы 3 месяца',
    'Опыт работы 6 лет'
])

def get_experience(arg):
    words_m = ['месяц', 'месяцев', 'месяца']
    words_y = ['лет', 'год', 'года']
    res = arg.split(' ')[2:]
    res.reverse()
    result = []
    for i in range(len(res)):
        if res[i] in words_m:
            result.append(int(res[i+1]))
        elif res[i] in words_y:
            result.append(int(res[i+1])*12)
    
    return sum(result)
print(test.apply(get_experience))
    